Notebook 9: Word2Vec
=======================================

## Goals for learning
In this assignment, we will:
1) Practice with the **[PyTorch](https://pytorch.org/)** library for working with neural networks.
2) Perform a deep-dive into the Word2Vec algorithm (CBOW variant).
3) Examine some applications of word embeddings.

## Instructions
* Read through the notebook.
* Answer any plain text questions (replace cell content, "YOUR RESPONSE HERE", with your response).
* Insert your code within the code blocks marked with the comments "# START your code here" and "# STOP your code here".
* Do not use any "Generative AI" tools or assistants in the creation of your solutions.
* Do not import or use any libraries other than those already imported outside the "# START your code here" and "# STOP your code here" blocks.
* Run all cells to make sure your code works and you see reasonable results.
    * All code cells should have output indicating the results of the last run when the notebook is submitted.
    * If there are errors, or if a code cell does not have output as submittted, points will be deducted.

## Submission details
* Due: Monday 11/17, 11:59 PM
* [Submission instructions](https://www.cs.oswego.edu/~agraci2/csc461/submission_instructions.html)

## Python library dependencies
* [Pytorch](https://pytorch.org/) - "PyTorch is an optimized tensor library for deep learning using GPUs and CPUs." 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

## Loading the data
The algorithm we are investigating today can take a long time to train, if given a large corpus to train from.

For this notebook, we will only be looking at a very small excerpt of text.

This excerpt was taken from ["The Bird Book" by Chester A. Reed (2009) via Project Gutenberg](https://www.gutenberg.org/cache/epub/30000/pg30000.txt).

In [2]:
raw_corpus = """Unlike the Grebes, Loons do not build in colonies, generally not more
than one, or at the most two pairs nesting on the same lake or pond;
neither do they seek the marshy sloughs in which Grebes dwell,
preferring the more open, clear bodies of water. The common Loon may be
known in summer by the entirely black head and neck with the complete
ribbon of black and white stripes encircling the lower neck and the
narrower one which crosses the throat. The back is spotted with white.
In some sections Loons build no nest, simply scooping a hollow out in
the sand, while in other places they construct quite a large nest of
sticks, moss and grasses. It is usually placed but a few feet from the
waters edge, so that at the least suspicion the bird can slide off its
eggs into the water, where it can cope with any enemy. The nests are
nearly always concealed under the overhanging bushes that line the
shore; the one shown in the full page illustration, however, was located
upon the top of an old muskrat house. The two eggs which they lay are a
very dark greenish brown in color, with black spots."""

## Cleaning our data
Before we learn from our data (corpus), we first want to perform a small amount of cleaning.

In the following "clean_corpus" function, perform the following transformations:
1) Convert all text to lower case.
2) Remove the following special symbols: ".", "--", ",", ";", ":", "_", "?", "!"
3) Remove the word "the" (note: this is a very common [stop word](https://en.wikipedia.org/wiki/Stop_word) to remove in natural language processing (NLP) tasks)

In [3]:
import re

In [4]:
def clean_corpus(corpus):
    # START your code here
    
    # 1
    corpus = corpus.lower()
    # print(corpus,'\n')
    
    # 2 - https://docs.python.org/3/library/re.html#
    corpus = re.sub(r'[^\w\s]', '', corpus)
    # print(corpus,'\n')

    # 3
    corpus = re.sub(r'the ', '', corpus)
    # print(corpus)
    # STOP your code here

    return corpus

corpus = clean_corpus(raw_corpus)
print(corpus)

# Simple checkpoint
# assert len(corpus) == 995, "Unexpected corpus length: {}".format(len(corpus)) 

unlike grebes loons do not build in colonies generally not more
than one or at most two pairs nesting on same lake or pond
neither do they seek marshy sloughs in which grebes dwell
preferring more open clear bodies of water common loon may be
known in summer by entirely black head and neck with complete
ribbon of black and white stripes encircling lower neck and the
narrower one which crosses throat back is spotted with white
in some sections loons build no nest simply scooping a hollow out in
sand while in other places they construct quite a large nest of
sticks moss and grasses it is usually placed but a few feet from the
waters edge so that at least suspicion bird can slide off its
eggs into water where it can cope with any enemy nests are
nearly always concealed under overhanging bushes that line the
shore one shown in full page illustration however was located
upon top of an old muskrat house two eggs which they lay are a
very dark greenish brown in color with black spots


## Building our vocabulary
The next thing we need to do is to build our vocabulary by **tokenizing** our cleaned corpus.
This means we break down our text into individual "tokens", which are usually words or phrases.

For this notebook, we are going to keep things simple and tokenize by splitting on  white space.

In [5]:
tokenized = None

'''
TODO: Split the cleaned corpus into tokens, using white space as a delimiter.
'''
# START your code here
tokenized = corpus.split()
# print(tokenized)
# STOP your code here

In [7]:
vocabulary = None

'''
TODO: Create a vocabulary by finding the set of all unique tokens.
'''
# START your code here
# https://www.geeksforgeeks.org/python/python-get-unique-values-list/
vocabulary = list(dict.fromkeys(tokenized))
# STOP your code here

vocabulary_size = len(vocabulary)
assert vocabulary_size == 134, "Unexpected vocabulary size: {}".format(vocabulary_size)
print(vocabulary_size)

134


## Encoding our text input
In order to train on our text data, we need a way to represent our text numerically.
In our lecture, we examined one-hot encoding for text data, but because we are working with such a small sample of data, and want to keep things simple, we will be encoding out text by simply mapping each word to a unique ID number.

In [8]:
word_to_id = {}
id_to_word = {}

'''
TODO: Create a map from each word in the vocabulary to a unique ID number, and another map from ID to word.
'''
# START your code here
ids = range(vocabulary_size)

word_to_id = dict(zip(vocabulary, ids))
id_to_word = dict(zip(ids, vocabulary))
# STOP your code here

assert isinstance(word_to_id, dict)
assert len(word_to_id) == vocabulary_size
assert isinstance(id_to_word, dict)
assert len(id_to_word) == vocabulary_size
print("Done.")

Done.


## Establishing our window of context words
When running Word2Vec, one of the parameters provided to you is the window size. 
This determines how much context to consider when processing each target word.

For example, a window size of 5 would look at 2 words context words before the target word, the target word, and 2 context words after. 

In [42]:
# # Emma test
# counter = 0
# WINDOW_SIZE = 5
# batches = []
# idx_first_full_window = int((WINDOW_SIZE - 1) / 2)
# print("idx_first_full_window: ", idx_first_full_window,'\n')
# idx_last_full_window = int(len(tokenized) - (WINDOW_SIZE - 1) / 2)
# print("idx_last_full_window: ", idx_last_full_window,'\n')
# for target_idx in range(idx_first_full_window, idx_last_full_window):
#     print('---------------------------------------------------------------------------\n',counter,'\n','---------------------------------------------------------------------------\n')
#     target = tokenized[target_idx]
#     print("target: ", target, '\n')
#     context = [tokenized[target_idx-1], tokenized[target_idx+1]]
#     print("context: ", context,'\n')
#     batches.append((context, target))
#     print("batches: ", batches, '\n')
#     counter += 1

# https://pythonguides.com/pytorch-dataloader/

In [10]:
WINDOW_SIZE = 5

def get_context_words(tokenized, target_id, WINDOW_SIZE):
    context = None
    
    '''
    TODO: Find the context words that come before and after the target word in the tokenized text.
    '''
    # START your code here
    half_window = (WINDOW_SIZE - 1) // 2
    context = tokenized[target_id - half_window : target_id] + \
              tokenized[target_id + 1 : target_id + half_window + 1]
    # STOP your code here
    
    assert len(context) == WINDOW_SIZE - 1
    return context

def batch_data(tokenized, WINDOW_SIZE):
    batches = []
    idx_first_full_window = int((WINDOW_SIZE - 1) / 2)
    idx_last_full_window = int(len(tokenized) - (WINDOW_SIZE - 1) / 2)
    for target_idx in range(idx_first_full_window, idx_last_full_window):
        target = tokenized[target_idx]
        context = get_context_words(tokenized, target_idx, WINDOW_SIZE)
        batches.append((context, target))
    return batches

input_batches = batch_data(tokenized, WINDOW_SIZE)
print("Example batch: {}".format(input_batches[5]))

Example batch: (['build', 'in', 'generally', 'not'], 'colonies')


Now that we have our input batches, we will need a utility function to facilitate converting our vector of context words into a vector of numeric IDs.

In [11]:
def context_words_to_ids(context, word_to_id):
    context_ids = None

    '''
    TODO: Convert context words to their IDs
    '''
    # START your code here
    context_ids = [word_to_id[word] for word in context] # mapping
    context_ids = torch.tensor(context_ids, dtype=torch.long)
    # STOP your code here
    
    return context_ids

example_batch_context_words = input_batches[5][0]
example_batch_context_ids = context_words_to_ids(example_batch_context_words, word_to_id)

# Simple checkpoint
assert isinstance(example_batch_context_ids, torch.Tensor)
assert len(example_batch_context_ids) == len(example_batch_context_words)

print(example_batch_context_words)
print(example_batch_context_ids)

['build', 'in', 'generally', 'not']
tensor([5, 6, 8, 4])


## Creating our CBOW model (Neural Network)
In the cell below, we will define our **neural network** model as a class called "CbowNet".

Using the PyTorch framework, neural network models are **subclasses** of the [**Module**](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) class.

> PyTorch uses modules to represent neural networks. Modules are:
>
> * Building blocks of stateful computation. PyTorch provides a robust library of modules and makes it simple to define new custom modules, allowing for easy construction of elaborate, multi-layer neural networks.
> * Tightly integrated with PyTorch’s autograd system. Modules make it simple to specify learnable parameters for PyTorch’s Optimizers to update.
> * Easy to work with and transform. Modules are straightforward to save and restore, transfer between CPU / GPU / TPU devices, prune, quantize, and more.
>
> From the [PyTorch v 2.1 Documentation](https://pytorch.org/docs/stable/notes/modules.html)

Implement the model below woth the following architecture:

1) An embedding (see: [nn.Embedding](https://docs.pytorch.org/docs/stable/generated/torch.nn.Embedding.html)) **hidden layer** with "EMBEDDING_SIZE" neurons.
   * Embedding layers are fully connected, like a linear layer, but are optimized for working with natural language data.
3) A summation of each of the (WINDOW_SIZE-1) embeddings for the context words (see: [torch.sum](https://docs.pytorch.org/docs/stable/generated/torch.sum.html)).
4) A linear **output layer** with "OUTPUT_SIZE" neurons that uses the log_softmax (see: [F.log_softmax](https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.log_softmax.html)) activation function.
   * Softmax can be used to convert a vector of raw outputs into a vector of probability scores.
   * We need to take the log softmax, because the loss function we will be using expects log to be applied.

In [15]:
'''
TO DO: Define the size of the input to the neural network and the size of the output from the neural network.
Hint: Review your lecture notes. What determines the size of the inputs and the size of the outputs?
'''
# START your code here
INPUT_SIZE = vocabulary_size
EMBEDDING_SIZE = 100
OUTPUT_SIZE = vocabulary_size
# STOP your code here

# Define the structure of the neural netowrk
class CbowNet(nn.Module):
    
    # Constructor
    def __init__(self):
        super(CbowNet, self).__init__()

        self.L1 = None # Hidden layer
        self.LN = None # Output layer
        
        '''
        TO DO: Instantiate the layers (L1-LN) according to the above description
        '''
        # START your code here
        self.L1 = nn.Embedding(INPUT_SIZE, EMBEDDING_SIZE)
        self.LN = nn.Linear(EMBEDDING_SIZE, OUTPUT_SIZE)
        # STOP your code here

    # Defines the steps taken during the "forward pass"
    def forward(self, data):
        '''
        TO DO: 
         - The input layer is provided above
         - Pass the data through the hidden and output layers, as defined above
        '''
        # START your code here
        embeds = self.L1(data)
        summed = torch.sum(embeds, dim=0)
        out = self.LN(summed) 
        out = F.log_softmax(out, dim=0)
        data = out.unsqueeze(0)  
        # STOP your code here
        
        return data

    # Performs a forward pass and returns the result with the highest probability
    def predict(self, data):
        probabilities = self.forward(data)
        return torch.argmax(probabilities).item()
        
    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_id[word]])
        return self.L1(word)

## Training our model

In [16]:
%%time
# ^This will display how long it takes to execute this cell

model = CbowNet()
loss_function = nn.NLLLoss()
LEARNING_RATE = 0.001
optimizer = torch.optim.SGD(model.parameters(), LEARNING_RATE)
EPOCHS = 500

for epoch in range(EPOCHS):
    total_error = 0

    for context, target in input_batches:
        X = context_words_to_ids(context, word_to_id)  
        y_t = torch.tensor([word_to_id[target]])
        y_p = model(X)
        total_error += loss_function(y_p, y_t)

    '''
    NOTE:
     - Here is where we start letting the optimizer calculate our derivitives for us!
    TO DO: 
     - Use your optimizer to zero-out the gradients for the weights and biases. Otherwise they would accumulate over each batch.
     - Using your total error function result, run the "backpropagation" or "backwards pass" through the model
        - Error goes from Output Layer -> Hidden Layer N -> ... -> Hidden Layer 1 -> Input Layer
     - Using your optimizer, Update the weights and biases
    '''
    # START your code here
    optimizer.zero_grad()        # Clear gradients
    total_error = total_error.unsqueeze(0)  # make it a 1×1 tensor to avoid size mismatch
    total_error.backward()       # backpropagate
    optimizer.step()             # update weights
    # STOP your code here

print("Done.")

Done.
CPU times: total: 2min 47s
Wall time: 55.2 s


## Application: Fill in the blank
One use of our model is to predict a target work based on a given context.

For our first prediction example, we will use a segment that exists in the input corpus: "ribon of *black* and white"

In [17]:
context_words = ['ribbon', 'of', 'and', 'white']
context_ids = context_words_to_ids(context_words, word_to_id)
y_p = model.predict(context_ids)
y_t = "black"

print('Context: {}'.format(context_words))
print('Predicted: {}'.format(id_to_word[y_p]))
print('Expected: {}'.format(y_t))

Context: ['ribbon', 'of', 'and', 'white']
Predicted: black
Expected: black


Next, we can give our model a novel context (which doesn't exist in the input corpus): "loons are _____ consealed under"

In [18]:
context_words = ['loons', 'are', 'concealed', 'under' ]
context_ids = context_words_to_ids(context_words, word_to_id)
y_p = model.predict(context_ids)

print('Context: {}'.format(context_words))
print('Predicted: {}'.format(id_to_word[y_p]))

Context: ['loons', 'are', 'concealed', 'under']
Predicted: always


## Finding synonyms
Next, we will be looking at another way we can use embeddings from our model's hidden layer: finding synonyms.

A synonym for a target word will be used in many of the same contexts as the target, therefore the learned embedding for both the target and its synonym will be close to eachother.

Because our embedding layer size is 100, there will be 100 learned weights for each word.
Let's take a look at what the embedding for the target word "loons" looks like, as an example.

In [19]:
example_word = "loons"
example_embedding = model.get_word_emdedding(example_word)
print("Example word: {}".format(example_word))
print("Embedding size: {}".format(example_embedding.size()))
print(example_embedding)

Example word: loons
Embedding size: torch.Size([1, 100])
tensor([[ 1.6877e+00, -2.4270e-01,  1.4254e+00, -3.6263e-01, -1.2473e-02,
         -6.1673e-01, -1.9608e+00,  6.5924e-01,  2.2071e+00, -7.4053e-02,
         -1.7828e+00, -7.7567e-01,  1.8969e+00,  9.2272e-01,  1.0088e+00,
         -9.6745e-02,  9.7054e-01, -2.4015e-01,  1.6926e+00,  1.7361e-02,
          1.2856e+00, -2.7721e-02, -4.9146e-02,  6.5882e-01,  1.2459e+00,
         -3.1179e-01, -1.7870e+00, -1.5852e+00, -1.0139e+00,  1.6499e+00,
         -1.3078e+00,  1.8393e-01, -2.5021e+00,  1.4890e+00,  7.9824e-01,
          7.7647e-01,  5.5561e-01,  1.2233e+00,  9.1133e-01, -1.6318e+00,
         -2.4021e+00,  7.9234e-01, -2.9244e-01,  1.4287e+00, -3.6585e-01,
         -4.2307e-02,  4.7193e-01,  1.3239e+00, -1.0924e-01,  1.4369e-01,
          1.4302e+00, -1.5916e+00,  2.3982e+00, -4.1991e-02, -1.8029e-01,
          7.6279e-01, -8.8968e-02,  2.2973e+00,  1.8521e+00, -3.2098e-01,
          6.8903e-04, -1.1925e+00, -6.3041e-01, -1.4235

One way to measure the "closeness" of these embeddings is by using a distance measure.

For simplicity and familiarity, we will be using the [Euclidean Distance](https://en.wikipedia.org/wiki/Euclidean_distance):

$$d(p,q)=\sqrt{\sum_i^{\text{features}}{(p_i - q_i)^2}}$$

Implementation hint: if you would like to work with embeddings as numpy nd-arrays instead of as tensors, you can use the following functions.
     * https://docs.pytorch.org/docs/stable/generated/torch.Tensor.detach.html
     * https://docs.pytorch.org/docs/stable/generated/torch.Tensor.numpy.html

In [20]:
import numpy

# Calculates the euclidean distance between two word embeddings
def Distance(target, other):
    distance = None
    
    '''
    TO DO: Implement the Euclidean Distance equation
    '''
    # START your code here
    distance = torch.sqrt(torch.sum((target.squeeze() - other.squeeze()) ** 2)).item()
    # STOP your code here
    
    return distance

In [23]:
# Calculates the word with the minimum distance from the target word
def MinDistance(target, others):
    min_idx = None

    '''
    TO DO: 
    1. Get all distances (from the target word to each of the other words)
    2. Get the index of the minimum distance
    '''
    # START your code here
    target_embedding = model.get_word_emdedding(target)
    other_embeddings = [model.get_word_emdedding(word) for word in others]
    
    distances = [Distance(target_embedding, other_emb) for other_emb in other_embeddings]
    min_idx = distances.index(min(distances))
    # STOP your code here
    
    return others[min_idx]

Use this last cell to change the target word to any word in our vocabulary. 
Experiment with different target words and see what synonyms our model comes up with. 

In [25]:
# START your code here
target_word = "always" # Choose any word from the vocabulary
# STOP your code here

# Find the closest synonym, using the minimum distance from all other vocabulary words
other_words = list(vocabulary.copy())
other_words.remove(target_word)
synonym = MinDistance(target_word, other_words)

print("Target word: {}".format(target_word))
print("Synonym: {}".format(synonym))

Target word: always
Synonym: known


Congratulations, you have reached the end of this notebook!